<a href="https://colab.research.google.com/github/DSLUCIFER/Network-intrusion-detection/blob/main/Copy_of_packets_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scapy
!pip install joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444328 sha256=fa41c8a43cd172201f6ae8e4c67610869fe056129a5f4f1a3d49769ccb8dbd01
  Stored in directory: /root/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy


In [ ]:
import pandas as pd
import numpy as np
from scapy.all import rdpcap, IP, TCP, UDP, Ether
from scapy.all import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
packets = rdpcap("/content/drive/MyDrive/CS/nids.pcapng")

In [ ]:
def extract_packet_features(packet):
    """
    Extracts features from a packet, handling potential missing values.

    Args:
        packet (scapy.Packet): The packet to extract features from.

    Returns:
        dict: A dictionary containing extracted features or placeholders for missing values.
    """

    features = {
        'source_ip': None,
        'destination_ip': None,
        'source_port': None,
        'destination_port': None,
        'protocol': None,
    }

    if IP in packet:
        features['source_ip'] = packet[IP].src
        features['destination_ip'] = packet[IP].dst
        features['protocol'] = packet[IP].proto

        if TCP in packet:
            features['source_port'] = packet[TCP].sport
            features['destination_port'] = packet[TCP].dport
        elif UDP in packet:
            features['source_port'] = packet[UDP].sport
            features['destination_port'] = packet[UDP].dport

        # Handle missing ports for non-TCP/UDP packets (optional)
        else:
            features['source_port'] = -1  # Example: placeholder for missing port
            features['destination_port'] = -1

    # Print extracted values for debugging (optional)
    print(f"Extracted features: {features}")

    return features

def pcap_to_dataframe(packets):
    """
    Converts a list of packets to a DataFrame with extracted features.

    Args:
        packets (list): A list of scapy packets.

    Returns:
        pd.DataFrame: A DataFrame containing extracted features.
    """

    features_list = [extract_packet_features(packet) for packet in packets if IP in packet]
    df = pd.DataFrame(features_list)

    # Check for missing values
    print(df.isnull().sum())

    return df

# Replace 'your_file.pcapng' with the actual file path

df = pcap_to_dataframe(packets)

# Display the extracted features
print(df.head())


Extracted features: {'source_ip': '192.168.254.9', 'destination_ip': '172.217.12.142', 'source_port': 52433, 'destination_port': 443, 'protocol': 17}
Extracted features: {'source_ip': '192.168.254.9', 'destination_ip': '172.217.12.142', 'source_port': 52433, 'destination_port': 443, 'protocol': 17}
Extracted features: {'source_ip': '192.168.254.9', 'destination_ip': '172.217.12.142', 'source_port': 52433, 'destination_port': 443, 'protocol': 17}
Extracted features: {'source_ip': '192.168.254.9', 'destination_ip': '172.217.12.142', 'source_port': 52433, 'destination_port': 443, 'protocol': 17}
Extracted features: {'source_ip': '192.168.254.9', 'destination_ip': '172.217.12.142', 'source_port': 52433, 'destination_port': 443, 'protocol': 17}
Extracted features: {'source_ip': '192.168.254.9', 'destination_ip': '172.217.12.142', 'source_port': 52433, 'destination_port': 443, 'protocol': 17}
Extracted features: {'source_ip': '172.217.12.142', 'destination_ip': '192.168.254.9', 'source_port'

In [ ]:
# Check for missing values
print(df.isnull().sum())

# If missing values are present, apply appropriate techniques as described earlier.


source_ip           0
destination_ip      0
source_port         0
destination_port    0
protocol            0
dtype: int64


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features_scaled = scaler.fit_transform(df[['source_port', 'destination_port']])  # Assuming these are numerical features
df[['source_port_scaled', 'destination_port_scaled']] = features_scaled  # Add scaled features as new columns


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler.fit_transform(df[['source_port', 'destination_port']])  # Assuming these are numerical features
df[['source_port_scaled', 'destination_port_scaled']] = features_scaled  # Add scaled features as new columns


In [ ]:

# Assuming "df" is your DataFrame containing the extracted features
# Replace "selected_features" with the list of feature names you want to use

selected_features = ["source_ip", "destination_ip", "source_port", "destination_port", "protocol"]  # Example

# Convert the selected features to a NumPy array
features_array = np.array(df[selected_features])

print("Extracted features as NumPy array:")
print(features_array)


Extracted features as NumPy array:
[['192.168.254.9' '172.217.12.142' 52433 443 17]
 ['192.168.254.9' '172.217.12.142' 52433 443 17]
 ['192.168.254.9' '172.217.12.142' 52433 443 17]
 ...
 ['20.198.119.143' '192.168.254.9' 443 49721 6]
 ['192.168.254.9' '20.198.119.143' 49721 443 6]
 ['192.168.254.9' '192.168.254.255' 138 138 17]]


In [ ]:
import joblib

In [ ]:
# Load the SVM model
model = joblib.load('svm_model.joblib')
